## sklearn-crfsuite

In this notebook we train a basic CRF model for Named Entity Recognition on CoNLL2002 data (following https://github.com/TeamHG-Memex/sklearn-crfsuite/blob/master/docs/CoNLL2002.ipynb) and check its weights to see what it learned.

In [1]:
import sys
sys.path.insert(0, '..')

import nltk
import sklearn_crfsuite

from eli5 import explain_weights, format_as_text
from eli5.formatters import fields

Load training data:

In [2]:
%%time
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

CPU times: user 2.71 s, sys: 97.2 ms, total: 2.8 s
Wall time: 2.81 s


Extract features: word parts, POS tags, lower/title/upper flags, features of nearby words

In [3]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],        
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [4]:
sent2features(train_sents[0])[0]

{'+1:postag': 'Fpa',
 '+1:postag[:2]': 'Fp',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': '(',
 'BOS': True,
 'bias': 1.0,
 'postag': 'NP',
 'postag[:2]': 'NP',
 'word.isdigit()': False,
 'word.istitle()': True,
 'word.isupper()': False,
 'word.lower()': 'melbourne',
 'word[-2:]': 'ne',
 'word[-3:]': 'rne'}

In [5]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

CPU times: user 1.63 s, sys: 158 ms, total: 1.78 s
Wall time: 1.81 s


Train a CRF model:

In [6]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1, 
    c2=0.1, 
    max_iterations=20, 
    all_possible_transitions=False,
)
crf.fit(X_train, y_train)

CPU times: user 14.3 s, sys: 219 ms, total: 14.5 s
Wall time: 14.6 s


Check CRF weights (transition weights and state weights):

In [7]:
expl = explain_weights(crf, top=20)
expl

Explanation(estimator="CRF(algorithm='lbfgs', all_possible_states=None,\n  all_possible_transitions=False, averaging=None, c=None, c1=0.1, c2=0.1,\n  calibration_candidates=None, calibration_eta=None,\n  calibration_max_trials=None, calibration_rate=None,\n  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,\n  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=20,\n  max_linesearch=None, min_freq=None, model_filename=None,\n  num_memories=None, pa_type=None, period=None, trainer_cls=None,\n  variance=None, verbose=False)", description=None, error=None, method='CRF', is_regression=False, targets=[TargetExplanation(target='B-LOC', feature_weights=FeatureWeights(pos=[('word.istitle()', 2.3082630000000002), ('-1:word.lower():en', 2.21529), ('word[-2:]:id', 0.96784000000000003), ('word[-3:]:rid', 0.92051000000000005), ('word.lower():madrid', 0.91972699999999996), ('word[-2:]:ia', 0.78586400000000001), ('word[-2:]:ña', 0.68621699999999997), ('word[-2:]:ís', 0.67472399999999999), ('word[-3:]:ona', 0.66492799999999996), ('word.lower():españa', 0.64580499999999996), ('-1:postag[:2]:Fp', 0.59740499999999996), ('-1:postag:Fpa', 0.58201800000000004), ('-1:word.lower():(', 0.58201800000000004), ('postag[:2]:NP', 0.56610000000000005), ('postag:NP', 0.56610000000000005)], neg=[('-1:word.istitle()', -1.3437209999999999), ('postag:SP', -1.011741), ('postag[:2]:SP', -1.011741), ('-1:word.lower():del', -0.83347599999999999), ('word[-3:]:ión', -0.625556)], pos_remaining=2428, neg_remaining=506), proba=None, score=None, weighted_spans=None), TargetExplanation(target='O', feature_weights=FeatureWeights(pos=[('postag[:2]:Fp', 3.643554), ('BOS', 3.5770309999999998), ('bias', 3.1395499999999998), ('postag:CC', 1.998208), ('postag[:2]:CC', 1.998208), ('postag[:2]:Fc', 1.8384199999999999), ('word[-3:]:,', 1.8384199999999999), ('postag:Fc', 1.8384199999999999), ('word.lower():,', 1.8384199999999999), ('word[-2:]:,', 1.8384199999999999), ('word.lower():y', 1.7064630000000001), ('EOS', 1.559275), ('postag:RG', 1.4515929999999999), ('postag[:2]:RG', 1.4515929999999999), ('word[-3:]:-', 1.278837), ('postag[:2]:Fg', 1.278837)], neg=[('word.istitle()', -6.5745909999999999), ('word.isupper()', -4.1145529999999999), ('postag[:2]:NP', -2.1962449999999998), ('postag:NP', -2.1962449999999998)], pos_remaining=16970, neg_remaining=4166), proba=None, score=None, weighted_spans=None), TargetExplanation(target='B-ORG', feature_weights=FeatureWeights(pos=[('word.lower():efe', 2.4133260000000001), ('word.isupper()', 2.1558670000000002), ('word[-2:]:FE', 1.1325320000000001), ('word[-3:]:EFE', 1.1172089999999999), ('word.lower():gobierno', 1.1032580000000001), ('-1:word.lower():del', 0.96011999999999997), ('word.istitle()', 0.88195299999999999), ('word[-3:]:rno', 0.84985699999999997), ('-1:word.lower():al', 0.77758499999999997), ('word[-2:]:PP', 0.74130200000000002), ('word.lower():pp', 0.73424999999999996), ('word[-3:]:PP', 0.73424999999999996), ('-1:postag[:2]:DA', 0.61978500000000003), ('-1:postag:DA', 0.61978500000000003)], neg=[('postag:SP', -1.417144), ('postag[:2]:SP', -1.417144), ('-1:word.lower():en', -0.73296300000000003), ('word[-2:]:es', -0.71601700000000001), ('-1:word.lower():de', -0.71592699999999998), ('-1:postag[:2]:VM', -0.68142100000000005)], pos_remaining=3848, neg_remaining=721), proba=None, score=None, weighted_spans=None), TargetExplanation(target='B-PER', feature_weights=FeatureWeights(pos=[('word.istitle()', 2.8049430000000002), ('postag[:2]:NP', 0.78541799999999995), ('postag:NP', 0.78541799999999995), ('+1:postag:VMI', 0.60152099999999997), ('-1:word.lower():a', 0.59059099999999998), ('+1:postag[:2]:VM', 0.59041100000000002)], neg=[('postag[:2]:DA', -1.165368), ('postag:DA', -1.165368), ('-1:word.istitle()', -1.024311), ('-1:postag[:2]:DA', -1.0027159999999999), ('-1:postag:DA', -1.0027159999999999), ('-1:word.lower():del', -0.85910299999999995), ('-1:word.lower():la', -0.74174700000000005), ('word[-2:]:os', -

It is also possible to format the result as text (could be useful in console):

In [8]:
print(format_as_text(expl))

Explained as: CRF

Transition features:
          B-LOC       O    B-ORG    B-PER    I-PER    B-MISC    I-ORG    I-LOC    I-MISC
------  -------  ------  -------  -------  -------  --------  -------  -------  --------
B-LOC    -0.060   0.030    0.000   -0.221    0.000     0.000    0.000    4.345     0.000
O         2.488   3.704    2.857    2.262    0.000     2.240    0.000    0.000     0.000
B-ORG    -0.217   0.265   -0.955   -0.381    0.000    -0.535    5.029    0.000     0.000
B-PER    -0.832  -0.336   -1.224   -1.169    4.528     0.000    0.000    0.000     0.000
I-PER    -0.277  -0.810    0.000   -0.728    3.611     0.000    0.000    0.000     0.000
B-MISC   -0.326  -0.651   -0.408   -0.354    0.000     0.000    0.000    0.000     5.870
I-ORG    -1.926  -0.420   -1.703   -0.622    0.000    -0.794    5.551    0.000     0.000
I-LOC    -0.608  -0.297    0.000    0.000    0.000     0.000    0.000    3.744     0.000
I-MISC   -0.986  -0.724   -0.929   -0.612    0.000    -0.421    0.000 